# Machine Learning Part 3: Neural Network

This file is load the data and see the result

In [1]:
import tensorflow as tf
import numpy as np

import numpy as np
import pandas as pd
import dicom
import os
import matplotlib.pyplot as plt
import cv2
import math

IMG_SIZE_PX = 50
SLICE_COUNT = 20

n_classes = 2

x = tf.placeholder('float')
y = tf.placeholder('float')

keep_rate = 0.8 # was 0.8


In [2]:
def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window as you slide about
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

def convolutional_neural_network(x):
    #                # 5 x 5 x 5 patches, 1 channel, 32 features to compute.
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,64])), # was 32, 64
               #       5 x 5 x 5 patches, 32 channels, 64 features to compute.
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,64,64])),
               #                                  64 features
               'W_conv3':tf.Variable(tf.random_normal([3,3,3,64,32])),
               #                                  64 features
               'W_conv4':tf.Variable(tf.random_normal([3,3,3,32,32])),
               'W_fc':tf.Variable(tf.random_normal([1024,1024])),
               'W_fc2':tf.Variable(tf.random_normal([1024,1024])),
               'W_fc3':tf.Variable(tf.random_normal([1024,1024])),# was 54080, 1024
               'out':tf.Variable(tf.random_normal([1024, n_classes]))} # was 1024

    biases = {'b_conv1':tf.Variable(tf.random_normal([64])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_conv3':tf.Variable(tf.random_normal([32])),
                'b_conv4':tf.Variable(tf.random_normal([32])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
              'b_fc2':tf.Variable(tf.random_normal([1024])),
              'b_fc3':tf.Variable(tf.random_normal([1024])),# was 16
               'out':tf.Variable(tf.random_normal([n_classes]))}

    #                            image X      image Y        image Z
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)


    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)
    
    conv3 = tf.nn.relu(conv3d(conv2, weights['W_conv3']) + biases['b_conv3'])
    conv3 = maxpool3d(conv3)
    
    conv4 = tf.nn.relu(conv3d(conv3, weights['W_conv4']) + biases['b_conv4'])
    conv4 = maxpool3d(conv4)
    print(conv4.get_shape)
    fc = tf.reshape(conv4,[-1, 1024])# was 54080
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc2'])+biases['b_fc2'])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc3'])+biases['b_fc3'])
    #fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output


In [3]:
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [4]:
#much_data = np.load('muchdata-50-50-20.npy')



#train_data = much_data[:-30]
#validation_data = much_data[-30:]
#train_data = much_data
# If you are working with the basic sample data, use maybe 2 instead of 100 here... you don't have enough data to really do this
much_data = np.load('muchdata-50-50-20-b.npy')
train_data = [[item[0],item[1]] for item in much_data]
test_data = train_data
#test_data = np.load('testdata-50-50-20-b.npy')
#test_data = [[item[0],item[1],item[2]] for item in test_data]

P_list = []
proba_list = []
proba_list_2 = []

def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=tf.to_int32(y) ))
    optimizer = tf.train.AdamOptimizer(learning_rate=3e-2).minimize(cost)
    
    hm_epochs = 300
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        successful_runs = 0
        total_runs = 0
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for data in train_data:
                total_runs += 1
                try:
                    X = data[0]
                    Y = data[1]
                    _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                    epoch_loss += c
                    successful_runs += 1
                except Exception as e:
                    # I am passing for the sake of notebook space, but we are getting 1 shaping issue from one 
                    # input tensor. Not sure why, will have to look into it. Guessing it's
                    # one of the depths that doesn't come to 20.
                    pass
                    #print(str(e))
            
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)

            #correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            #accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

            #for i in xrange(10):
            #    val_data = chunkIt(validation_data,10)[i]
            #    print('Accuracy:',accuracy.eval({x:[i[0] for i in val_data], y:[i[1] for i in val_data]}))
            #print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
            
            #print accuracy
            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            predictions = tf.argmax(prediction,1)
            proba = tf.cast(prediction, 'float')
            proba_2 = tf.nn.softmax(logits=prediction)
                        
        print('Done. Finishing accuracy:')
        for j in xrange(30):
            te_data = chunkIt(test_data,30)[j]
            print('Accuracy:',accuracy.eval({x:[i[0] for i in te_data], y:[i[1] for i in te_data]}))
            P_list.append(predictions.eval({x: [i[0] for i in te_data]}))
            proba_list.append(proba.eval({x: [i[0] for i in te_data]}))
            proba_list_2.append((proba_2.eval({x: [i[0] for i in te_data]})))
            
        #print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
        print('fitment percent:',successful_runs/total_runs)

# Run this locally:
train_neural_network(x)

<bound method Tensor.get_shape of <tf.Tensor 'MaxPool3D_3:0' shape=(?, 4, 4, 2, 32) dtype=float32>>
Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Epoch', 1, 'completed out of', 300, 'loss:', 18255361176363.68)
('Epoch', 2, 'completed out of', 300, 'loss:', 80394663774.758057)
('Epoch', 3, 'completed out of', 300, 'loss:', 18794388512.245117)
('Epoch', 4, 'completed out of', 300, 'loss:', 1183221271.0177066)
('Epoch', 5, 'completed out of', 300, 'loss:', 38672.649850113972)
('Epoch', 6, 'completed out of', 300, 'loss:', 45506.837578773499)
('Epoch', 7, 'completed out of', 300, 'loss:', 49666.856366779539)
('Epoch', 8, 'completed out of', 300, 'loss:', 44119.535979270891)
('Epoch', 9, 'completed out of', 300, 'loss:', 30466.789261988219)
('Epoch', 10, 'completed out of', 300, 'loss:', 35887.674167070298)
('Epoch', 11, 'completed out of', 300, 'loss:', 46474.258435249329)
('Epoch', 12, 'completed out of', 300, 'loss:', 43257.01299814877)
('Epoch', 13, 'comple

('Epoch', 116, 'completed out of', 300, 'loss:', 1068.1582178410863)
('Epoch', 117, 'completed out of', 300, 'loss:', 1090.8878818145354)
('Epoch', 118, 'completed out of', 300, 'loss:', 1000.922125542408)
('Epoch', 119, 'completed out of', 300, 'loss:', 694.33995292774182)
('Epoch', 120, 'completed out of', 300, 'loss:', 717.09522112546733)
('Epoch', 121, 'completed out of', 300, 'loss:', 836.54985472136968)
('Epoch', 122, 'completed out of', 300, 'loss:', 745.64614597445052)
('Epoch', 123, 'completed out of', 300, 'loss:', 694.00889497708772)
('Epoch', 124, 'completed out of', 300, 'loss:', 593.70308358438911)
('Epoch', 125, 'completed out of', 300, 'loss:', 584.70056853627852)
('Epoch', 126, 'completed out of', 300, 'loss:', 585.28741755687747)
('Epoch', 127, 'completed out of', 300, 'loss:', 442.38354610628016)
('Epoch', 128, 'completed out of', 300, 'loss:', 587.18603985251775)
('Epoch', 129, 'completed out of', 300, 'loss:', 563.10322142093889)
('Epoch', 130, 'completed out of', 

('Epoch', 235, 'completed out of', 300, 'loss:', 179.46112886816263)
('Epoch', 236, 'completed out of', 300, 'loss:', 179.46058963425457)
('Epoch', 237, 'completed out of', 300, 'loss:', 179.45997444353998)
('Epoch', 238, 'completed out of', 300, 'loss:', 179.45916377753019)
('Epoch', 239, 'completed out of', 300, 'loss:', 179.45824052579701)
('Epoch', 240, 'completed out of', 300, 'loss:', 179.45707878284156)
('Epoch', 241, 'completed out of', 300, 'loss:', 179.45577521808445)
('Epoch', 242, 'completed out of', 300, 'loss:', 179.45421543903649)
('Epoch', 243, 'completed out of', 300, 'loss:', 179.45231319218874)
('Epoch', 244, 'completed out of', 300, 'loss:', 179.45007910765707)
('Epoch', 245, 'completed out of', 300, 'loss:', 179.44739052653313)
('Epoch', 246, 'completed out of', 300, 'loss:', 179.44427710585296)
('Epoch', 247, 'completed out of', 300, 'loss:', 179.44059344381094)
('Epoch', 248, 'completed out of', 300, 'loss:', 179.43637744523585)
('Epoch', 249, 'completed out of',

In [5]:
feature_test = [np.sum(img[0]) for img in test_data]

In [6]:
labels = pd.read_csv('../../cap2input/stage2_solution.csv', index_col=0)
labels.cancer.value_counts()

0    353
1    153
Name: cancer, dtype: int64

In [7]:
proba_list_2

[array([[ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366]], dtype=float32),
 array([[ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366]], dtype=float32),
 array([[ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366],
        [ 0.84299636,  0.15700366]], dtype=float32),
 array([[ 0.84299636,  0.15700366],
        [ 0.8

In [8]:
P_list[0]

array([0, 0, 0, 0, 0, 0, 0, 0])

In [9]:
ori_list = [item for sublist in P_list for item in sublist]
print ori_list

true_list = [item[1][1] for item in test_data]
print true_list

patient_list = [item[2] for item in test_data]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

IndexError: list index out of range

In [ ]:
ori_list_proba = [item for sublist in proba_list_2 for item in sublist]

ori_list_proba_true = [[-sublist[1],-sublist[0]] if (sublist[0]+sublist[1]<0) else sublist for sublist in ori_list_proba]


proba_list_true = [sublist/sum(sublist) for sublist in ori_list_proba_true]

proba_list_0 = [sublist[0] for sublist in proba_list_true]
proba_list_1 = [0.1 if sublist[1]<0.1 else 1 if sublist[1]>1 else sublist[1] for sublist in proba_list_true]

#ori_list_proba = [np.exp(item) for sublist in proba_list for item in sublist]

print proba_list_1

In [ ]:
#ori_list_proba_2 = [item for sublist in proba_list_2 for item in sublist]
#print ori_list_proba_2

In [ ]:
labels['proba'] = proba_list_1
labels['predictions'] = ori_list
labels['true'] = true_list
labels['feature'] = feature_test
labels['patient'] = patient_list


In [20]:
pred = labels[['patient','proba']].reset_index(drop=True)
pred.columns = ['id','cancer']

In [21]:
#pred[pred == 0] = 0.4
#pred[pred == 1] = 0.6

In [22]:
pred

,id,cancer
0,7a632e7b8309e4b57a5964762a9e594a,0.239519
1,d662c2884bf723e2b0b2401ccab08887,0.182052
2,edde23221a8c4038f9a10daa3088ee94,0.239519
3,91541739b4a106e5481a6156c5b3cfbb,0.239519
4,d82af7a99fb96b2df37b4a0db270795f,0.182052
5,2b76a32e5c12e4312cb029721a24b190,0.239519
6,307a08ece9a33b772c903f5afbf9a799,0.239519
7,5d669e1f9abdff1a282b924c828a9b0a,0.260963
8,ad690e1d0df849ee7983f69ab43e785c,0.187214
9,c6c77fd425da262d6eeed53cff38cd91,0.187214


In [23]:
pred.to_csv('new_fourth_sub.csv', index=False)